In [1]:
# Chatbot Trắc Nghiệm VNHSGE - Lý, Hóa, Sinh
# Dataset: https://github.com/Xdao85/VNHSGE

In [2]:
import pandas as pd
import numpy as np
import json
import re
import random
import os
import glob
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import pickle
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

In [3]:
# PHẦN 1: XỬ LÝ DỮ LIỆU

class DataProcessor:
    def __init__(self):
        self.subjects = ['Biology', 'Chemistry', 'Physics']
        self.subject_map = {
            'Biology': 'Sinh',
            'Chemistry': 'Hóa', 
            'Physics': 'Lý'
        }
        
    def load_vnhsge_data(self, data_folder: str = 'Dataset') -> pd.DataFrame:
        """Load dữ liệu từ thư mục Dataset"""
        all_data = []
        
        for subject in self.subjects:
            subject_path = os.path.join(data_folder, subject)
            
            if not os.path.exists(subject_path):
                print(f"  Không tìm thấy thư mục: {subject_path}")
                continue
                
            # Tìm tất cả file JSON trong thư mục môn học
            json_files = glob.glob(os.path.join(subject_path, "*.json"))
            
            subject_count = 0
            for json_file in json_files:
                try:
                    with open(json_file, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                    
                    # Xử lý từng câu hỏi
                    for item in data:
                        if 'Question' in item and 'Choice' in item:
                            # Tách câu hỏi và đáp án từ trường Question
                            question_text, options = self._parse_question(item['Question'])
                            
                            question_data = {
                                'id': item.get('ID', ''),
                                'question': question_text,
                                'options': options,
                                'answer': item['Choice'],
                                'subject': subject.lower(),
                                'explanation': item.get('Explanation', ''),
                                'image_question': item.get('Image_Question', ''),
                                'image_answer': item.get('Image_Answer', '')
                            }
                            all_data.append(question_data)
                            subject_count += 1
                            
                except Exception as e:
                    print(f" Lỗi khi đọc file {json_file}: {e}")
                    continue
            
            print(f"📚 {self.subject_map[subject]}: {subject_count} câu hỏi")
        
        if not all_data:
            print(" Không tìm thấy dữ liệu nào!")
            return pd.DataFrame()
            
        return pd.DataFrame(all_data)
    
    def _parse_question(self, question_full: str) -> Tuple[str, List[str]]:
        """Tách câu hỏi và các đáp án từ text"""
        lines = question_full.split('\n')
        
        # Dòng đầu là câu hỏi (bỏ "Câu XX:")
        question = lines[0]
        if question.startswith('Câu'):
            question = re.sub(r'^Câu \d+:\s*', '', question)
        
        # Các dòng tiếp theo là đáp án
        options = []
        for line in lines[1:]:
            line = line.strip()
            if line and (line.startswith('A.') or line.startswith('B.') or 
                        line.startswith('C.') or line.startswith('D.')):
                options.append(line)
        
        return question.strip(), options
    
    def preprocess_text(self, text: str) -> str:
        """Tiền xử lý văn bản"""
        # Chuyển về chữ thường
        text = text.lower()
        # Loại bỏ các ký tự đặc biệt, giữ lại tiếng Việt
        text = re.sub(r'[^\w\sàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]', ' ', text)
        # Loại bỏ khoảng trắng thừa
        text = ' '.join(text.split())
        return text

In [ ]:
# TRAINING MODEL

class QuestionClassifier:
    def __init__(self):
        self.vectorizer = TfidfVectorizer(
            max_features=5000, 
            stop_words=None,
            ngram_range=(1, 2),  # Sử dụng cả unigram và bigram
            min_df=2,  # Loại bỏ từ xuất hiện quá ít
            max_df=0.95  # Loại bỏ từ xuất hiện quá nhiều
        )
        self.classifier = MultinomialNB(alpha=0.1)
        self.is_trained = False
        
    def train(self, X_train: List[str], y_train: List[str], X_test: List[str], y_test: List[str]):
        """Train model phân loại câu hỏi theo môn học"""
        print(" Bắt đầu training model...")
        
        # Vectorize dữ liệu
        X_train_vec = self.vectorizer.fit_transform(X_train)
        X_test_vec = self.vectorizer.transform(X_test)
        
        # Train model
        self.classifier.fit(X_train_vec, y_train)
        
        # Đánh giá model
        y_pred = self.classifier.predict(X_test_vec)
        accuracy = accuracy_score(y_test, y_pred)
        
        print(f"  KẾT QUẢ TRAINING:")
        print(f"   - Số câu hỏi train: {len(X_train)}")
        print(f"   - Số câu hỏi test: {len(X_test)}")
        print(f"   - Accuracy: {accuracy:.3f}")
        print(f"   - Số features: {X_train_vec.shape[1]}")
        
        print("\n Chi tiết theo môn học:")
        report = classification_report(y_test, y_pred, target_names=['biology', 'chemistry', 'physics'])
        print(report)
        
        self.is_trained = True
        return accuracy
    
    def predict_subject(self, question: str) -> str:
        """Dự đoán môn học của câu hỏi"""
        if not self.is_trained:
            return 'unknown'
        
        processed_question = DataProcessor().preprocess_text(question)
        question_vec = self.vectorizer.transform([processed_question])
        prediction = self.classifier.predict(question_vec)[0]
        return prediction
    
    def save_model(self, path: str):
        """Lưu model"""
        model_data = {
            'vectorizer': self.vectorizer,
            'classifier': self.classifier,
            'is_trained': self.is_trained
        }
        with open(path, 'wb') as f:
            pickle.dump(model_data, f)
        print(f" Model đã được lưu tại: {path}")
    
    def load_model(self, path: str):
        """Load model"""
        try:
            with open(path, 'rb') as f:
                model_data = pickle.load(f)
            self.vectorizer = model_data['vectorizer']
            self.classifier = model_data['classifier']
            self.is_trained = model_data['is_trained']
            print(f" Model đã được load từ: {path}")
            return True
        except FileNotFoundError:
            print(f" Không tìm thấy file model: {path}")
            return False

In [5]:
# CHATBOT
class VNHSGEChatbot:
    def __init__(self):
        self.processor = DataProcessor()
        self.classifier = QuestionClassifier()
        self.data = None
        self.current_subject = None
        self.current_question = None
        self.score = {'correct': 0, 'total': 0}
        
    def setup(self, data_folder: str = 'Dataset', model_file: str = 'vnhsge_classifier.pkl'):
        """Khởi tạo chatbot"""
        print(" Khởi tạo Chatbot...")
        
        # Load dữ liệu từ thư mục Dataset
        self.data = self.processor.load_vnhsge_data(data_folder)
        
        if self.data.empty:
            print("Không có dữ liệu để khởi tạo chatbot!")
            return False
            
        print(f"Đã load tổng cộng {len(self.data)} câu hỏi")
        
        # Thống kê dữ liệu
        print("\n THỐNG KÊ DỮ LIỆU:")
        subject_counts = self.data['subject'].value_counts()
        for subject, count in subject_counts.items():
            subject_name = self.processor.subject_map.get(subject.title(), subject)
            print(f"   - {subject_name}: {count} câu")
        
        # Training hoặc load model
        if not self.classifier.load_model(model_file):
            self._train_model()
            self.classifier.save_model(model_file)
        
        print("\n Chatbot đã sẵn sàng!")
        return True
        
    def _train_model(self):
        """Train model nếu chưa có"""
        print("\n Training model mới...")
        
        # Chuẩn bị dữ liệu training
        questions = []
        subjects = []
        
        for _, row in self.data.iterrows():
            # Kết hợp câu hỏi và đáp án để có nhiều thông tin hơn
            full_text = row['question'] + ' ' + ' '.join(row['options']) if row['options'] else row['question']
            processed_question = self.processor.preprocess_text(full_text)
            questions.append(processed_question)
            subjects.append(row['subject'])
        
        # Chia train/test (80/20) với stratified sampling
        X_train, X_test, y_train, y_test = train_test_split(
            questions, subjects, test_size=0.2, random_state=42, stratify=subjects
        )
        
        # Training
        accuracy = self.classifier.train(X_train, y_train, X_test, y_test)
        
    def start_chat(self):
        """Bắt đầu chat"""
        print("\n" + "="*60)
        print("🎓 CHATBOT TRẮC NGHIỆM VNHSGE")
        print(" Các môn: Lý - Hóa - Sinh (2019-2023)")
        print("="*60)
        
        while True:
            try:
                if not self.current_subject:
                    self._choose_subject()
                else:
                    self._ask_question()
                    
            except KeyboardInterrupt:
                self._show_final_score()
                print("\n👋 Tạm biệt! Cảm ơn bạn đã sử dụng chatbot.")
                break
            except Exception as e:
                print(f" Có lỗi xảy ra: {e}")
                continue
    
    def _choose_subject(self):
        """Chọn môn học"""
        print(f"\n📊 Điểm hiện tại: {self.score['correct']}/{self.score['total']}")
        if self.score['total'] > 0:
            accuracy = (self.score['correct'] / self.score['total']) * 100
            print(f"🎯 Độ chính xác: {accuracy:.1f}%")
            
        print("\n🎯 Chọn môn học:")
        print("1. Lý (Physics)")
        print("2. Hóa (Chemistry)")  
        print("3. Sinh (Biology)")
        print("4. Ngẫu nhiên")
        print("5. Xem thống kê")
        print("0. Thoát")
        
        choice = input("\nNhập lựa chọn (0-5): ").strip()
        
        if choice == '0':
            self._show_final_score()
            print(" Tạm biệt!")
            exit()
        elif choice == '1':
            self.current_subject = 'physics'
        elif choice == '2':
            self.current_subject = 'chemistry'
        elif choice == '3':
            self.current_subject = 'biology'
        elif choice == '4':
            self.current_subject = random.choice(['physics', 'chemistry', 'biology'])
        elif choice == '5':
            self._show_statistics()
            return
        else:
            print(" Lựa chọn không hợp lệ!")
            return
        
        subject_name = self.processor.subject_map[self.current_subject.title()]
        print(f"\n Bạn đã chọn môn: {subject_name}")
        
        # Kiểm tra có câu hỏi không
        subject_questions = self.data[self.data['subject'] == self.current_subject]
        if len(subject_questions) == 0:
            print(" Không có câu hỏi nào cho môn này!")
            self.current_subject = None
        
    def _show_statistics(self):
        """Hiển thị thống kê chi tiết"""
        print("\n THỐNG KÊ CHI TIẾT:")
        for subject in ['biology', 'chemistry', 'physics']:
            subject_data = self.data[self.data['subject'] == subject]
            subject_name = self.processor.subject_map[subject.title()]
            print(f"\n{subject_name}:")
            print(f"   - Tổng số câu: {len(subject_data)}")
            
            # Thống kê theo năm
            years = {}
            for _, row in subject_data.iterrows():
                if 'ID' in row and row['ID']:
                    year = re.search(r'(\d{4})', row['ID'])
                    if year:
                        year = year.group(1)
                        years[year] = years.get(year, 0) + 1
            
            for year in sorted(years.keys()):
                print(f"     + Năm {year}: {years[year]} câu")
        
        input("\nNhấn Enter để tiếp tục...")
        
    def _ask_question(self):
        """Đặt câu hỏi"""
        # Lấy câu hỏi ngẫu nhiên theo môn
        subject_questions = self.data[self.data['subject'] == self.current_subject]
        self.current_question = subject_questions.sample(1).iloc[0]
        
        # Hiển thị câu hỏi
        print(f"\n {self.current_question['question']}")
        
        # Hiển thị các đáp án
        if self.current_question['options']:
            for option in self.current_question['options']:
                print(f"   {option}")
        
        # Hiển thị thông tin thêm nếu có
        if self.current_question['image_question']:
            print(f"  Có hình ảnh kèm theo: {self.current_question['image_question']}")
        
        # Nhận câu trả lời
        print("\n Nhập đáp án hoặc lệnh:")
        print("   A/B/C/D - Chọn đáp án")
        print("   'back' - Đổi môn")
        print("   'hint' - Xem gợi ý")
        
        answer = input("Lựa chọn của bạn: ").strip().upper()
        
        if answer == 'BACK':
            self.current_subject = None
            return
        elif answer == 'HINT':
            self._show_hint()
            return
        elif answer in ['A', 'B', 'C', 'D']:
            self._check_answer(answer)
        else:
            print(" Vui lòng nhập A, B, C, D, 'back' hoặc 'hint'!")
    
    def _show_hint(self):
        """Hiển thị gợi ý"""
        # Dự đoán môn học từ câu hỏi
        predicted_subject = self.classifier.predict_subject(self.current_question['question'])
        actual_subject = self.current_question['subject']
        
        print(f"\n💡 GỢI Ý:")
        print(f"   - Model dự đoán đây là câu hỏi môn: {self.processor.subject_map.get(predicted_subject.title(), predicted_subject)}")
        print(f"   - Môn thực tế: {self.processor.subject_map.get(actual_subject.title(), actual_subject)}")
        
        if predicted_subject == actual_subject:
            print("   Model dự đoán đúng!")
        else:
            print("    Model dự đoán sai!")
            
        input("\nNhấn Enter để tiếp tục...")
    
    def _check_answer(self, user_answer: str):
        """Kiểm tra câu trả lời"""
        correct_answer = self.current_question['answer'].upper()
        self.score['total'] += 1
        
        if user_answer == correct_answer:
            self.score['correct'] += 1
            print("ĐÚNG!")
        else:
            print(f"SAI! Đáp án đúng là: {correct_answer}")
        
        # Hiển thị giải thích
        if self.current_question['explanation']:
            print(f"💡 Giải thích: {self.current_question['explanation']}")
        
        # Hiển thị thông tin câu hỏi
        if self.current_question['id']:
            print(f"ID câu hỏi: {self.current_question['id']}")
        
        # Hiển thị điểm
        accuracy = (self.score['correct'] / self.score['total']) * 100
        print(f" Điểm: {self.score['correct']}/{self.score['total']} ({accuracy:.1f}%)")
        
        input("\nNhấn Enter để tiếp tục...")
    
    def _show_final_score(self):
        """Hiển thị điểm cuối"""
        if self.score['total'] > 0:
            accuracy = (self.score['correct'] / self.score['total']) * 100
            print(f"\n🏆 KẾT QUẢ CUỐI CÙNG:")
            print(f"   - Tổng câu hỏi: {self.score['total']}")
            print(f"   - Số câu đúng: {self.score['correct']}")
            print(f"   - Độ chính xác: {accuracy:.1f}%")
            
            if accuracy >= 80:
                print(" Xuất sắc!")
            elif accuracy >= 60:
                print(" Khá tốt!")
            elif accuracy >= 40:
                print(" Cần ôn tập thêm!")
            else:
                print(" Hãy cố gắng hơn!")

In [6]:
# MAIN EXECUTION

In [ ]:
def main():
    """Chạy chatbot"""
    chatbot = VNHSGEChatbot()
    
    # Khởi tạo với thư mục Dataset
    if chatbot.setup(data_folder='Dataset'):
        chatbot.start_chat()
    else:
        print("Không thể khởi tạo chatbot. Vui lòng kiểm tra thư mục Dataset.")

if __name__ == "__main__":
    main()

 Khởi tạo Chatbot...
📚 Sinh: 200 câu hỏi
📚 Hóa: 200 câu hỏi
📚 Lý: 200 câu hỏi
Đã load tổng cộng 600 câu hỏi

 THỐNG KÊ DỮ LIỆU:
   - Sinh: 200 câu
   - Hóa: 200 câu
   - Lý: 200 câu
 Không tìm thấy file model: vnhsge_classifier.pkl

 Training model mới...
 Bắt đầu training model...
  KẾT QUẢ TRAINING:
   - Số câu hỏi train: 480
   - Số câu hỏi test: 120
   - Accuracy: 0.992
   - Số features: 3761

 Chi tiết theo môn học:
              precision    recall  f1-score   support

     biology       1.00      0.97      0.99        40
   chemistry       1.00      1.00      1.00        40
     physics       0.98      1.00      0.99        40

    accuracy                           0.99       120
   macro avg       0.99      0.99      0.99       120
weighted avg       0.99      0.99      0.99       120

 Model đã được lưu tại: vnhsge_classifier.pkl

 Chatbot đã sẵn sàng!

🎓 CHATBOT TRẮC NGHIỆM VNHSGE
 Các môn: Lý - Hóa - Sinh (2019-2023)

📊 Điểm hiện tại: 0/0

🎯 Chọn môn học:
1. Lý (Physics)
2.